In [43]:
import numpy as np
from functools import reduce

In [27]:
import mxnet as mx

In [28]:
from mnist import MNIST
#you can find python-mnist source code on https://github.com/sorki/python-mnist

datahandler = MNIST('../mnist_data_loader') #change for data path
train_data = datahandler.load_training()

In [29]:
image_array = np.asarray(train_data[0])

In [30]:
label_list = []
for i in train_data[1]:
    l = [0]*10
    l[i] = 1
    label_list.append(l)

label_array = np.asarray(label_list)

In [51]:
train_finish = 50000
eval_finish = 55000
predict_start = 55000

In [32]:
train_iter = mx.io.NDArrayIter(
    
    data = image_array[:train_finish],
    label = label_array[:train_finish],
    batch_size = 100,

)

In [33]:
val_iter = mx.io.NDArrayIter(
    
    data = image_array[train_finish:eval_finish],
    label = label_array[train_finish:eval_finish],
    batch_size = 100,

)

In [34]:
data = mx.symbol.Variable('data')
fc1  = mx.symbol.FullyConnected(data = data, num_hidden=784)
act1 = mx.symbol.Activation(data = fc1, act_type="sigmoid")
fc2  = mx.symbol.FullyConnected(data = act1, num_hidden=10)
mlp  = mx.symbol.SoftmaxOutput(data = fc2, name = 'softmax')

In [35]:
model = mx.model.FeedForward(
    
    symbol = mlp,
    num_epoch = 10,
    learning_rate = .01

)

model.fit(X = train_iter, eval_data = val_iter)

In [57]:
test_iter = mx.io.NDArrayIter(
    
    data = image_array[predict_start:],
    batch_size = 1,

)

In [37]:
model.predict(X = test_iter)

array([[  3.04934474e-06,   6.21107165e-06,   2.38730308e-06,
          4.74400586e-05,   2.14421598e-04,   3.49034271e-05,
          1.24247606e-06,   1.05475087e-03,   8.16530010e-05,
          9.98553932e-01],
       [  1.17014002e-04,   6.50333895e-05,   9.94722962e-01,
          7.11755129e-05,   3.17175261e-04,   9.68343375e-05,
          4.37919796e-03,   5.36119342e-06,   2.22135946e-04,
          3.17442755e-06],
       [  2.05261713e-05,   5.34496212e-05,   8.14498526e-06,
          4.33851295e-04,   2.17523091e-04,   2.80854380e-04,
          1.99101055e-06,   8.03659018e-03,   1.84345030e-04,
          9.90762770e-01],
       [  3.17822851e-04,   6.72896713e-05,   2.38706198e-04,
          6.25139568e-04,   1.79747934e-04,   9.97237802e-01,
          1.28923240e-03,   4.01389980e-06,   3.45748158e-05,
          5.56843952e-06],
       [  5.17457966e-06,   9.89887655e-01,   2.17162305e-03,
          5.33406215e-04,   2.03845455e-04,   6.36984827e-04,
          2.17914145e-04

In [38]:
print(label_array[predict_start:])

[[0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]]


In [39]:
p = model.predict(X = test_iter)
for j in range(0, len(p)):
    data_index = predict_start + j
    result = 'Inconclusive'
    one = p[j]
    for i in range(0,10):
        if one[i] > 0.99:
            label = list(filter(lambda i: label_array[data_index][i], range(0,10)))[0]
            result = 'predicted {} and it was {}'.format(i, label)
            print(datahandler.display(image_array[data_index]))
    print(result)


............................
............................
............................
............................
............................
............................
.................@@@........
...............@@@.@........
.............@@.............
............@...............
..........@.................
.........@@........@@@......
.........@.....@@@@@@.......
.........@@@@@....@@........
.................@@.........
................@@..........
...............@@...........
...............@............
..............@.............
.............@..............
............@@..............
...........@................
..........@.................
.........@@.................
........@@..................
........@...................
............................
............................
predicted 9 and it was 9

............................
............................
............................
............................
..................@@@@......
................

# Metrics

### Matriz de confusión
Útil para los cálculos

In [58]:
p = model.predict(X = test_iter)

In [59]:
len(p)

5000

In [63]:
conf_mtx = np.zeros([10, 10])
for j in range(0, len(p)):
    prediction = p[j]
    max_pred = reduce(lambda x, y: (x if prediction[x] > prediction[y] else y), range(0,10))
    actual_label = list(filter(lambda i: label_array[predict_start + j][i], range(0,10)))[0]
    conf_mtx[max_pred][actual_label] += 1
print(conf_mtx)
    

[[ 477.    0.    2.    0.    1.    2.    0.    0.    1.    0.]
 [   0.  526.    0.    1.    4.    0.    0.    2.    3.    1.]
 [   2.    1.  480.    2.    0.    2.    0.    1.    2.    1.]
 [   2.    0.    2.  498.    0.    8.    0.    2.    6.    4.]
 [   1.    0.    2.    0.  486.    1.    0.    3.    0.    5.]
 [   0.    0.    1.    4.    0.  434.    1.    0.    6.    2.]
 [   2.    1.    1.    0.    1.    8.  479.    0.    0.    0.]
 [   2.    0.    2.    0.    1.    1.    0.  553.    1.    2.]
 [   1.    2.    3.    2.    0.    1.    2.    1.  472.    1.]
 [   2.    0.    0.    2.    6.    1.    0.    1.    3.  467.]]


### Accuracy

In [72]:
acc = (sum(conf_mtx[i][i] for i in range(0, 10))/5000)

In [119]:
print('Accuracy: {:.5f}'.format(acc))

Accuracy: 0.97440


### Recall

In [87]:
recall = lambda i: (conf_mtx[i][i]/sum(conf_mtx[j][i] for j in range(0,10)))

In [120]:
recall_sum = 0
for i in range(0,10):
    rcl = recall(i)
    recall_sum += rcl
    print('Recall {}: {:.5f}'.format(i, rcl))
print()
print('Recall mean: {:.5f}'.format(recall_sum/10))

Recall 0: 0.97546
Recall 1: 0.99245
Recall 2: 0.97363
Recall 3: 0.97839
Recall 4: 0.97395
Recall 5: 0.94760
Recall 6: 0.99378
Recall 7: 0.98224
Recall 8: 0.95547
Recall 9: 0.96687

Recall mean: 0.97398


### Precision

In [93]:
precision = lambda i: (conf_mtx[i][i]/sum(conf_mtx[i][j] for j in range(0,10)))

In [118]:
precision_sum = 0
for i in range(0,10):
    label_precision = precision(i)
    precision_sum += label_precision
    print('Precision {}: {:.5f}'.format(i, label_precision))
print()
print('Precision mean: {:.5f}'.format(precision_sum/10))

Precision 0: 0.98758
Precision 1: 0.97952
Precision 2: 0.97760
Precision 3: 0.95402
Precision 4: 0.97590
Precision 5: 0.96875
Precision 6: 0.97358
Precision 7: 0.98399
Precision 8: 0.97320
Precision 9: 0.96888

Precision mean: 0.97430
